We will consider the following five simple baseline models to provide foundational insights. These will serve as the baseline performance metrics that any subsequent machine learning models aim to beat.

1. **Global Mean Rating**:
This model predicts the global mean rating for all user-item pairs, serving as the most basic form of recommendation.

2. **User Mean Rating**:
For this model, the mean rating of each user is calculated and used to predict ratings for all items the user has not yet interacted with.

3. **Item Mean Rating**:
In contrast to the User Mean Rating, this model focuses on the mean rating of each item and uses it to predict ratings for all users.

4. **User-Item Mean Rating**:
This model takes a more nuanced approach by predicting a rating for a user-item pair as the average of the user's mean rating and the item's mean rating. The formula is:
$$prediction = \frac{User Mean Rating + Item Mean Rating}{2}$$

5. **Weighted Mean Ratings**:
This model employs a weighted average of the user mean and item mean ratings. The weight ( w ) can be adjusted based on domain understanding. The formula is :
$$prediction = w \times User Mean Rating + (1 - w) \times Item Mean Rating, \space where \space 0 \leq w \leq 1$$

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
movies_df = pd.read_csv("../data/movies_metadata_after_eda.csv")
ratings_df = pd.read_csv("../data/ratings_small.csv")

In [3]:
train_df, test_df = train_test_split(ratings_df, test_size=0.3, random_state=42)

In [4]:
from Sameer.ModelExperimentation import (
    calculate_user_item_mean_rating,
    calculate_weighted_mean_ratings,
    calculate_rmse,
)

test_df = calculate_user_item_mean_rating(train_df, test_df)
test_df, best_w, best_rmse = calculate_weighted_mean_ratings(test_df)
test_df.head()

,userId,movieId,rating,timestamp,global_mean_rating,user_mean_rating,item_mean_rating,user_item_mean_rating,weighted_mean_rating
0,128,1028,5.0,1049690908,3.540256,3.844444,3.836364,3.840404,3.841212
1,665,4736,1.0,1010197684,3.540256,3.294304,3.540256,3.417280,3.392685
2,120,4002,3.0,1167420604,3.540256,3.573684,3.318182,3.445933,3.471483
3,257,1274,4.0,1348544094,3.540256,3.801370,3.791667,3.796518,3.797489
4,468,6440,4.0,1296191715,3.540256,2.946196,3.900000,3.423098,3.327717


In [5]:
test_df["constant_rating"] = 2.5

user_stats = train_df.groupby("userId")["rating"].agg(["mean", "count"]).reset_index()

DAMPING_FACTOR = 0.2

test_df = test_df.merge(user_stats, on="userId", how="left")
test_df["damped_user_mean_rating"] = ((1 - DAMPING_FACTOR) * test_df["mean"]) + (
    DAMPING_FACTOR * test_df["global_mean_rating"][0]
)

rmse_damped_user_mean = calculate_rmse(
    test_df["rating"], test_df["damped_user_mean_rating"]
)

In [6]:
rmse_global_mean = calculate_rmse(test_df["rating"], test_df["global_mean_rating"])
rmse_user_mean = calculate_rmse(test_df["rating"], test_df["user_mean_rating"])
rmse_item_mean = calculate_rmse(test_df["rating"], test_df["item_mean_rating"])
rmse_user_item_mean = calculate_rmse(
    test_df["rating"], test_df["user_item_mean_rating"]
)
rmse_weighted_mean = calculate_rmse(test_df["rating"], test_df["weighted_mean_rating"])
rmse_constant = calculate_rmse(test_df["rating"], test_df["constant_rating"])
rmse_damped_user_mean = calculate_rmse(
    test_df["rating"], test_df["damped_user_mean_rating"]
)

In [7]:
print(f"RMSE for Global Mean Rating Model: {rmse_global_mean:.2f}")
print(f"RMSE for User Mean Rating Model: {rmse_user_mean:.2f}")
print(f"RMSE for Item Mean Rating Model: {rmse_item_mean:.2f}")
print(f"RMSE for User-Item Mean Rating Model: {rmse_user_item_mean:.2f}")
print(
    f"RMSE for Weighted Mean Rating Model: {best_rmse:.2f}, with setting the best weight is {best_w:.2f}"
)
print(f"RMSE for Constant Model: {rmse_constant:.2f}")
print(f"RMSE for Damped User-Mean Rating Model: {rmse_damped_user_mean:.2f}")

RMSE for Global Mean Rating Model: 1.06
RMSE for User Mean Rating Model: 0.96
RMSE for Item Mean Rating Model: 1.00
RMSE for User-Item Mean Rating Model: 0.92
RMSE for Weighted Mean Rating Model: 0.92, with setting the best weight is 0.60
RMSE for Constant Model: 1.49
RMSE for Damped User-Mean Rating Model: 0.96


In building a movie recommendation system, we start with simple models due to it's importance in understanding how well we're doing. We found out that using both the average ratings from users and movies gets the best results, with a score (RMSE) of 0.92. This score shows how accurate our predictions are – the lower, the better. Models that only look at users or movies are okay, but not as good. The worst model is the one that always guesses the same rating, with a score of 1.49. These simple models help us know what to beat; any new, fancier model should score lower than 0.92 to be better.

We also look at more complex models because the simple ones don't consider what each user likes. Machine learning models can give more personalized suggestions, which is better for users.